In [1]:
%load_ext autotime
import requests
import bs4
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import re

import json

time: 714 ms (started: 2021-09-29 17:26:34 +07:00)


In [2]:
username = '518H0003'
passw = ''

time: 548 µs (started: 2021-09-29 17:26:36 +07:00)


In [4]:
session = requests.Session()
response = session.post("https://stdportal.tdtu.edu.vn/Login/SignIn?ReturnURL=https://stdportal.tdtu.edu.vn/",
                        data={'user': username, 'pass': passw}, cookies=session.cookies.get_dict())

print(session.cookies.get_dict())

ssoUrl=json.loads(response.text)['url']

print(ssoUrl)

token = re.findall('(?<=Token=)([^&]*)(?=&)?',ssoUrl)
print(token)

{'.AspNetCore.Cookies': 'CfDJ8CSm8XR270xNp5XK-6N6VqnMUnW_Pv436l3_BmdQkFNWYYb8PjPaVunCB3m9tZQoTvjmVYkJw0i8muulEhFnwzEPIDMjwMmy44sihCRU9z-fW0jSlLmvzo6s9VdLeGqFN1E92Ye5Ini-uqztx06wDBRd_Iqv2YM4T-2D5wWprZ4mfodMirX5waHTrV1UlSO0soFSj1GmyE3i5q9K9ibxB2vHkjezeouf5AlXeUpnB5YKVzRxMht2BNFWFxw21033k6-cDzTmJjW1REeECFH_Nv8R74iszy4jW3ndq73J8GpvBMZp1AXA_k4ASBsz51vqFcWaQGlQ69dyODKflowgagK1vUIqaVVgEAic9yOAm1_-Xd6M85E22N2m955vV0GOafc-VvPDam57LFDFOchCS69-g1fx_Pj5F66v23D81iTStuE1-XSdgnBzvkpylmmdYf_rR3gZUVahvkJngD_wrq-Xka3TxfRIvx-NODK3XQGRPLgUhE2wvR_a6z_DjSNtbQLmnneFjPkiUr8vs0qrME9b67wWaD3lrFPOpXCHeh6BA64F2kXnv5GkblGqK7EiZ4MJmL8kbaFwjOrlq15mGh4SuzKlEcBRlutcQN5R_HbzfnK7SiGLFDVg2dtHiQ-r0WoaylwH3jOKxqplhMJIyxLCYWtR4Z41Oeeo445aD32i_EraL2Mm-XbOyLe8FCtdWtRqa015ZpLPFLAT4GZ_ZAcWOJNYsBrnYws_2gEDtHLFQSWrnprps37gAFV0kvoNDMfGLA-Y5DPQAjL0d4-sydADvRrEamlMluFUtI9sVnWdna7N9jUcLVqGOkpjYL6_PuD8L5NrfQq8k4mfU4Cgzf-G0bX_vzJabq6zjF-fTCcEzFj9bZsURI4AwhNl2FGi1M3Kn99VHxCFv4pAvtHDPDGwHtzyeOoxZbbAf9nVAt1m6uv45NXLWgcvgey5nWJTYsRXhM73naw

In [260]:
resUrl = 'http://sso.tdt.edu.vn/Authenticate.aspx?ReturnUrl=https://lichhoc-lichthi.tdtu.edu.vn/tkb2.aspx&Token='+token[0]
response = requests.get(resUrl)

time: 432 ms (started: 2021-09-28 09:07:19 +07:00)


In [261]:
soup = BeautifulSoup(response.content, 'xml')
# soup.select('/html/body/form/div[3]/div[4]/table/tbody/tr/td[2]/select/option[5]')
table = soup.find('table', id='ThoiKhoaBieu1_tbTKBTheoTuan')
table

<table border="1" cellpadding="10" cellspacing="0" id="ThoiKhoaBieu1_tbTKBTheoTuan" rules="all" width="100%">
<tr class="Headerrow">
<td width="30"><font face="Tahoma">Tiết|Thứ <br/>Period | Day</font></td><td><font face="Tahoma">Thứ 2 | Monday<br/>(27/09)</font></td><td><font face="Tahoma">Thứ 3 | Tuesday<br/>(28/09)</font></td><td><font face="Tahoma">Thứ 4 | Wednesday<br/>(29/09)</font></td><td><font face="Tahoma">Thứ 5 | Thursday<br/>(30/09)</font></td><td><font face="Tahoma">Thứ 6 | Friday<br/>(01/10)</font></td><td><font face="Tahoma">Thứ 7 | Saturday<br/>(02/10)</font></td><td><font face="Tahoma">Chủ nhật | Sunday<br/>(03/10)</font></td>
</tr><tr class="rowContent">
<td class="cellbuoi"><font face="Tahoma">1</font></td><td class="cell"><font face="Tahoma"/></td><td class="cell"><font face="Tahoma"/></td><td bgcolor="#D94270" class="cell" rowspan="3"><font color="White" face="Tahoma"><table>100% cellpadding=0 cellspacing=0 &gt;<tr><td><b>Nhập môn Xử lý ảnh số<br/><label class="lbl

time: 64.3 ms (started: 2021-09-28 09:07:21 +07:00)


In [263]:
rawheads = table.findAll('tr', {'class': 'Headerrow'})
heads = rawheads[0].findAll('td')

rawarrHead = []
for i in range(1, len(heads)):
    rawarrHead.append(heads[i].text)

arrHeads = []
for i in rawarrHead:
    arrHeads.append(re.findall(r'\((.*)\)', i))

arrHeads = np.array(arrHeads).ravel()

time: 3.32 ms (started: 2021-09-28 09:08:13 +07:00)


In [264]:
trs = table.findAll('tr', {'class': 'rowContent'})
arrItems = np.zeros((len(trs), len(arrHeads)), dtype='U256')
for tr in range(len(trs)):
    tds = trs[tr].findAll('td', {'class': 'cell'})

    for td in range(len(tds)):
        tmp = tds[td].text
        tmpValue = re.findall(r'>(.*)', tmp)
        if len(tmpValue) > 0:
            arrItems[tr, td] = tmpValue[0]

time: 6.43 ms (started: 2021-09-28 09:08:15 +07:00)


In [265]:
dfSchedule = pd.DataFrame(arrItems, columns=arrHeads)
dfSchedule

,27/09,28/09,29/09,30/09,01/10,02/10,03/10
0,,,Nhập môn Xử lý ảnh số|Introduction to Digital ...,,Xử lý dữ liệu lớn|Massive Data Processing Tech...,,
1,,,,,,,
2,,,,,,,
3,Nhập môn Trí tuệ nhân tạo|Introduction to Arti...,,,,,,
4,,,,,,,
5,,,,,,,
6,,,,,,,
7,,,,,,,
8,,,,,,,
9,,Thực hành Nhập môn Xử lý ảnh số|Thực hành Intr...,,,,,


time: 23.1 ms (started: 2021-09-28 09:08:16 +07:00)


In [266]:
with open('NguyenQuocCuong-518H0003/Output/BSSchedule.txt', 'w+') as f:
    for i in dfSchedule.columns:
        f.write(f'Ngay thang: {i} \n')
        if (dfSchedule[i].sum() == ''):
            f.write(f'Lich cong tac: Khong co \n')
        else:
            for j in dfSchedule[i]:
                if (j != ''):
                    f.write(f'Lich cong tac: {j} \n')
        f.write(f'\n')

time: 6.88 ms (started: 2021-09-28 09:08:20 +07:00)
